<a href="https://colab.research.google.com/github/haru1489248/nlp-100-nock/blob/main/ch10/section_94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 94. チャットテンプレート
“What do you call a sweet eaten after dinner?”という問いかけに対する応答を生成するため、チャットテンプレートを適用し、言語モデルに与えるべきプロンプトを作成せよ。また、そのプロンプトに対する応答を生成し、表示せよ。

### チャットテンプレートとは？
会話形式で書いたやり取りを、モデルが実際に読むプロンプトに変換するための定型フォーマットのこと

### modelのrole
- system: ルール
- user: 質問
- assistant: 生成されるもの

In [43]:
from huggingface_hub import notebook_login
notebook_login()

In [44]:
!pip install -U transformers

In [45]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [46]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"
chat = [
    {"role": "system", "content": "You are a friendly chatbot who always responds in the style of a pirate"},
    {"role": "user", "content": "What do you call a sweet eaten after dinner?"},
]
max_new_tokens = 50

In [47]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id)

### tokenizer.apply_chat_templateとは？
chat形式（role付きの辞書）をモデルが読める形式に変換する関数
- tokenize=False(デフォルト): これは文字列（プロンプトを返す）
```
<|system|> You are a pirate
<|user|> What do you call...
<|assistant|>
```
  -  Trueの場合はトークン化して返す
### add_generation_prompt=Trueとは？
assistantが追加されてモデルが生成したものを出力できる様になる

In [49]:
tokenize_chat = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, return_tensors="pt")
prompt_len = tokenize_chat["input_ids"].shape[-1]

In [50]:
generation_config = GenerationConfig(
    max_new_tokens=max_new_tokens,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

In [51]:
output = model.generate(**tokenize_chat, generation_config=generation_config)

入力も返されるので、プロンプトの長さでマスクする

In [52]:
shifted_sentence = output[0][prompt_len:]
print(shifted_sentence)

tensor([ 9014,   637,    11, 20043,   387,  2610,   258,     6,   264,  7060,
         3488,    11, 30276,    88,     0, 32269,   387,  1781,   258,     6,
          297,     6,   264, 10437,  4322,  1306,   264, 82651, 14177,    11,
        36346,    30,  8489,    11,   358,   387,  2019,   258,     6, 20043,
          387,  1781,   258,     6,   297,     6,  1131,  9362,  3472,    11])


In [53]:
sentence = tokenizer.decode(shifted_sentence, skip_special_tokens=True)

In [54]:
print(sentence)

Arrrr, ye be askin' a fine question, matey! Ye be thinkin' o' a sweet treat after a hearty dinner, eh? Well, I be sayin' ye be thinkin' o'... CAKE,
